<a href="https://colab.research.google.com/github/tatyanach/work-projects/blob/main/%D0%A1%D0%BE%D0%BF%D0%BE%D1%81%D1%82%D0%B0%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D0%9D%D0%A2%D0%94_%D0%B8%D0%B7_%D0%95%D0%A1%D0%9C_%D0%B8_%D0%A2%D0%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, zipfile
import regex as re
import pandas as pd
from tqdm import tqdm


In [ ]:
def split_docs(df, column):
    '''Разделяет строку в датафрейме по запятой или точке запятой. 
    На вход нужен датафрейм и название колонки'''
    new_list = []
    for i, row in df.iterrows():
        doc_name = row[column]   

        for sep in [';', ',']:
            if sep in doc_name:
                gs = doc_name.split(sep)
                df.at[i, column] = gs[0]
                
                new_list.extend(gs[1:])
    df = df.append(pd.DataFrame(new_list, columns=[column]))
    return df

In [ ]:
def change_values(df, column, pat, repl):
    '''Заменяет все значения по шаблону на другое значение (добавляет пробелы). 
    На вход: датафрейм, название колонки, регулярное выражение и шаблон,
    на который нужно заменить значения с таким выражением'''
    for i, row in df.iterrows():
        doc_name = row[column]
        new_doc = re.sub(pat, repl, doc_name)
        df.at[i, column] = new_doc
    return df

In [ ]:
def match_completed(df1, column1, df2, column2):
    df1["te_exist"] = 0
    for i, row in tqdm(df1.iterrows(), total=df1.shape[0], position=0, leave=True):
        if len(df2[df2[column2] == row[column1]]) > 0:
            df1.at[i, "te_exist"] = 1    

In [ ]:
def match_string(df1, column1, df2, column2):
    '''Сопоставляет НД из ЕСМ с НД из ТЭ.
    df1 - датафрейм из есм,
    column1 - название для новой колонки в df1. колонка показывает наличие совпадения с ТЭ.
    df2 - датафрейм из ТЭ,
    column2 - название колонки из df2 с тайтлами, содержащими НД, которые нужно сопоставить'''
    df1['te_contains'] = 0
    for i, row in tqdm(df1.iterrows(), total=df1.shape[0], position=0, leave=True):
#         print(row[column1])
        if df2[column2].str.contains(row[column1], regex=False).any():
            df1.at[i, 'te_contains'] = 1
    return df1

In [ ]:
def show_result(df, column):
    
    not_f = len(df[df[column] == 0])
    found = len(df[df[column] == 1])
    print(f"Найдено {found} совпадений из {len(df)}, {not_f} документов не найдено в ТехЭксперте")

**ГОСТы из ТехЭксперта** предобработка

In [ ]:
df = pd.read_csv("te_docs.txt", sep = "\t")
df

,#,id,title
0,0,6185a0f48ef9e78dd1e85a6d,Об Учебно-методическом объединении по профорие...
1,1,6185a0f78ef9e78dd1e85b49,О присвоении имени музыкального деятеля Н.Г.Ру...
2,2,6185a0f78ef9e78dd1e85b4a,О создании государственного учреждения культур...
3,3,6185a0f78ef9e78dd1e85b4b,ГОСТ Р 22.8.07-99 Безопасность в чрезвычайных ...
4,4,6185a0f78ef9e78dd1e85b4c,ГОСТ 16372-93 (МЭК 34-9-90) Машины электрическ...
...,...,...,...
2751822,2751822,61861e4d8ef9e78dd1555d7b,Суд кассационной инстанции не усматривает осно...
2751823,2751823,61861e4d8ef9e78dd1555d7d,При новом рассмотрении дела суду необходимо уч...
2751824,2751824,61861e4d8ef9e78dd1555d80,Учитывая установленные в результате проведения...
2751825,2751825,61861e4d8ef9e78dd1555d81,Дело N 33-1334/2021


сначала отфильтруем таблицу и оставим только те строки, которые содержат в себе слово "ГОСТ"

In [ ]:
df_te = df[df["title"].str.contains('ГОСТ', regex=False)]


In [ ]:
# поиск гостов с помощью регулярок. для match_complete()
# df_te["gost"] = df_te['title'].apply(lambda x: re.search(r"ГОСТ\s?(Р\s)?([А-Я]+\s)?\d+([\d.-]+)?", x).group(0) 
#                                      if re.search(r"ГОСТ\s?(Р\s)?([А-Я]+\s)?\d+([\d.-]+)?", x) else None)
# df_te = df_te[df_te['gost'].notnull()]

In [ ]:
df_te

,#,id,title
3,3,6185a0f78ef9e78dd1e85b4b,ГОСТ Р 22.8.07-99 Безопасность в чрезвычайных ...
4,4,6185a0f78ef9e78dd1e85b4c,ГОСТ 16372-93 (МЭК 34-9-90) Машины электрическ...
7,7,6185a0f78ef9e78dd1e85b50,ГОСТ 5422-73 Профили стальные горячекатаные сп...
8,8,6185a0f78ef9e78dd1e85b51,ГОСТ 3240.0-76 Сплавы магниевые. Общие требова...
9,9,6185a0f78ef9e78dd1e85b52,ГОСТ 25278.2-82 Сплавы и лигатуры редких метал...
...,...,...,...
2751791,2751791,61861e4d8ef9e78dd1555d51,Проект ГОСТ Р Ресурсосбережение. Общие подходы...
2751795,2751795,61861e4d8ef9e78dd1555d56,Проект ГОСТ Р Ресурсосбережение. Информационно...
2751798,2751798,61861e4d8ef9e78dd1555d5b,Проект ГОСТ Р Ресурсосбережение. Информационно...
2751802,2751802,61861e4d8ef9e78dd1555d5f,Проект ГОСТ Р Ресурсосбережение. Информационно...


In [ ]:
#проверяем есть ли госты без пробелов в записи
df_te[df_te["title"].str.contains(r"ГОСТ\d+")]

,#,id,title
1076307,1076307,6185c3b38ef9e78dd1be316f,Является ли ГОСТ12.0.004-2015 обязательным для...
2237284,2237284,6185dc4a8ef9e78dd1512b6a,"ГОСТ16363-98 ""Огнезащитные составы"""
2698076,2698076,61860a458ef9e78dd1c2cd50,О применении ГОСТ30732-2020
2728817,2728817,618618228ef9e78dd12ab80e,О допустимости замены труб по ГОСТ20295-85


In [ ]:
#добавляем пробелы
df_te = change_values(df_te, "title", r"ГОСТ(\d+)", "ГОСТ \1")

In [ ]:
df_te[df_te["title"].str.contains(r"ГОСТ\d+")] #проверяем

,#,id,title


**НД из ЕСМ** - предобработка

In [ ]:
gosts = pd.read_excel("ГОСТы.xlsx")

разделяем строчки, в которых указано по два НД и записываем каждый НД в отдельную строчку

In [ ]:
# new_list = []
# for i, row in gosts.iterrows():
#     doc_name = row["Нормативный документ"]   
    
#     for sep in [';', ',']:
#         if sep in doc_name:
#             gs = doc_name.split(sep)
#             gosts.at[i, "Нормативный документ"] = gs[0]
# #             gosts.loc[len(gosts)] = [gs[1]]
#             new_list.append(gs[1])
# gosts = gosts.append(pd.DataFrame(new_list, columns=['Нормативный документ']))

In [ ]:
gosts = split_docs(gosts, "Нормативный документ")
gosts[gosts["Нормативный документ"].str.contains(r";|,")] #проверяем

,Нормативный документ


добавляем пробел к тем записям ГОСТов, которые записаны без пробела

In [ ]:
# 1 способ
# del_spaces = gosts[gosts["Нормативный документ"].str.contains(r"ГОСТ\d+")]["Нормативный документ"].tolist()
# for i, row in gosts.iterrows():
#     doc_name = row["Нормативный документ"]
#     if doc_name in del_spaces:
#         new_doc = doc_name[:4] + " " + doc_name[4:]
#         gosts.at[i, "Нормативный документ"] = new_doc
# 2 способ
# for i, row in gosts.iterrows():
#     doc_name = row["Нормативный документ"]
#     if re.search(r"ГОСТ\d+", doc_name):
#         new_doc = doc_name[:4] + " " + doc_name[4:]
#         gosts.at[i, "Нормативный документ"] = new_doc



In [ ]:
gosts[gosts["Нормативный документ"].str.contains(r"ГОСТ\d+")]

,Нормативный документ
101,ГОСТ17378-83
197,ГОСТ12.2.007.0-75
414,ГОСТ13781.2-77
747,ГОСТ19904-90
754,ГОСТ2590-88
755,ГОСТ19903-74
756,ГОСТ8560-78
761,ГОСТ8239-72
767,ГОСТ22580-88
787,ГОСТ8607-82


In [ ]:
gosts = change_values(gosts, "Нормативный документ", r"ГОСТ(\d+)", "ГОСТ \1")

In [ ]:
# for i, row in gosts.iterrows():
#     doc_name = row["Нормативный документ"]
#     new_doc = re.sub(r"ГОСТ(\d+)", "ГОСТ \1", doc_name)
#     gosts.at[i, "Нормативный документ"] = new_doc

проверяем наналичие записей без пробелов

In [ ]:
gosts[gosts["Нормативный документ"].str.contains(r"ГОСТ\d+")]

,Нормативный документ


убираем дубли

In [ ]:
gosts.dropna(inplace=True)

In [ ]:
gosts

,Нормативный документ
0,ГОСТ 2590-88
1,ГОСТ 4543-71
2,ГОСТ 2590-88
3,ГОСТ 1050-88
4,ГОСТ 14-1-5228-93
...,...
124,ГОСТ 11677-85
125,ГОСТ 17375-01
126,ГОСТ 8.586-2005
127,ГОСТ 30732-2001


**СОПОСТАВЛЕНИЕ**

1 способ - точное совпадение (находит меньше документов, но вероятность совпадения 100%)

In [ ]:
match_completed(gosts, "title", df_te, "title")
# show_result(gosts, "te_exist")


100%|██████████████████████████████████████████████████████████████████████████████| 1138/1138 [09:03<00:00,  2.09it/s]


In [ ]:
gosts[gosts["te_exist"] == 0]

,ГОСТ,title,te_exist,te_contains
0,ГОСТ 4543-71,"""ГОСТ 4543-71 Прокат из легированной конструкц...",0,0
5,ГОСТ 52630-2006,"""ГОСТ Р 52630-2006 Сосуды и аппараты стальные ...",0,0
9,ГОСТ 20714-75,"""ГОСТ 20714-75 Ткани фильтровальные хлорирован...",0,0
15,ГОСТ МЭК 60598-1-2003,"""ГОСТ Р МЭК 60598-1-2003 Светильники. Часть 1....",0,0
19,ГОСТ 7396-1-89,"""ГОСТ 7396.1-89 (МЭК 83-75) Соединители электр...",0,0
...,...,...,...,...
1118,ОСТ 2 Г93-20-87,"""ОСТ 2 Г93-20-87 Штуцера под манометры. Констр...",0,0
1120,ОСТ 6-01-37-88,"""ОСТ 6-01-37-88 Смола поливинилхлоридная хлори...",0,0
1123,ОСТ 32.134-99,"""ОСТ 32.134-99 Брусья железобетонные предварит...",0,0
1128,ОСТ 37.003.051-81,"""ОСТ 37.003.051-81 Фары противотуманные. Техни...",0,0


2 способ - поиск по подстроке (оптимальный, находит также разные версии одного и того же НД)

In [ ]:
match_string(gosts, "title", df_te, "title")
show_result(gosts, "te_contains")

100%|████████████████████████████████████████████████████████████████████████████████| 262/262 [10:29<00:00,  2.40s/it]

Найдено 0 совпадений из 262, 262 документов не найдено в ТехЭксперте


In [ ]:
not_found = gosts[gosts['te_contains'] == 0]
not_found.to_excel("Not_found_GOSTs.xlsx")


тестирование:

In [ ]:
df_te[df_te["title"].str.contains("ГОСТ 33259-2015")]["title"].tolist()

['Поправка к ГОСТ 33259-2015 Фланцы арматуры, соединительных частей и трубопроводов на номинальное давление до PN 250. Конструкция, размеры и общие технические требования',
 'Поправка к ГОСТ 33259-2015 Фланцы арматуры, соединительных частей и трубопроводов на номинальное давление до PN 250. Конструкция, размеры и общие технические требования',
 'Об опечатке в ГОСТ 33259-2015 "Фланцы арматуры, соединительных частей и трубопроводов на номинальное давление до PN 250. Конструкция, размеры и общие технические требования"',
 'ГОСТ 33259-2015 Фланцы арматуры, соединительных частей и трубопроводов на номинальное давление до PN 250. Конструкция, размеры и общие технические требования (с Поправками)',
 'Каких рекомендаций (ГОСТ 32569-2013 или ГОСТ 33259-2015) следует придерживаться при проектировании технологических трубопроводов, в частности при выборе типа уплотнительной поверхности фланцев?',
 'При определении типа уплотнительных поверхностей фланцев согласно приложения "Р" ГОСТ 32569-2013 и 